In [34]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from urllib.parse import urlparse
from collections import deque
import time
import random
import os
import pandas

In [3]:
HEADER = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36 Edg/143.0.0.0",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://www.google.com/"}


session = requests.Session()
session.headers.update(HEADER)

In [4]:
Filters_MBG = [r'/products?/', r'/shop/', r'/search?', r'/store/', r'/cart', r'/checkout', r'/account', 
           r'/login', r'/register', r'/tag', r'/search', r'#', "youtube.com", 
           "instagram.com", "facebook.com", "twitter.com", "pinterest.com", "linkedin.com", "tiktok.com", 
           "amazon.com", "google.com"]

In [5]:
def fetch_verify_url(url) :
    try:
        response = session.get(url, timeout=10)
        if response.status_code != 200:  # If the status code is not OK (200) the function return none and an error message
            print(f"Failed to fetch the url: {url} with status code {response.status_code}")
            return None
        return response  
    except requests.RequestException:
        return None

In [6]:
def to_soup(url):
    response = fetch_verify_url(url)
    if response:  # If the response is not none, the function return the beautiful soup object
        return BeautifulSoup(response.text, 'html.parser') 
    else:
        return None

In [25]:
def extract_MBG_links(url_MBG, max_levels=3):
    queue = deque([(url_MBG, 0)])
    links_MBG = set([url_MBG])
    start_domain = urlparse(url_MBG).netloc


    while queue:
        current_url, current_level = queue.popleft()

        # Si on atteint le niveau max, on arrête de creuser à partir d'ici
        if current_level >= max_levels:
            continue
    
        soup = to_soup(current_url)
        if not soup:
            continue

        content_area = soup.find('div', id='content') or soup.body
        if not content_area:
            continue

        for item in content_area.find_all('a', href=True):
            href = item.get('href')
            full_url = urljoin(current_url, href)
                    
            # Si nouveau lien trouvé
            if any(filter in full_url for filter in Filters_MBG):
                continue
            
            if full_url not in links_MBG:
                links_MBG.add(full_url)

                # On ajoute à la queue SEULEMENT si c'est interne (pour continuer le crawl)
                if urlparse(full_url).netloc == start_domain:
                    queue.append((full_url, current_level + 1))
                    
    return links_MBG

ALL_MBG_LINKS = extract_MBG_links("https://www.mindbodygreen.com/")
print(ALL_MBG_LINKS)
print(len(ALL_MBG_LINKS))

{'https://www.mindbodygreen.com/articles/7-sleep-mistakes-everyone-makes-and-how-to-fix-them', 'https://www.mindbodygreen.com/sponsor-disclaimer', 'https://www.mindbodygreen.com/articles/menstrual-cycle-workout', 'https://www.mindbodygreen.com/articles/when-your-brain-begins-to-atrophy-and-what-to-do-from-expert-what-to-to-about-it', 'https://www.mindbodygreen.com/articles/3-types-of-stress-and-how-to-manage-them', 'https://www.mindbodygreen.com/articles/viscose-fabric', 'https://www.mindbodygreen.com/articles/this-treatment-can-reverse-cognitive-decline-and-heal-brain-injury', 'https://www.mindbodygreen.com/wellness-trends', 'https://www.mindbodygreen.com/articles/this-common-leisure-activity-may-increase-dementia-risk-for-better-alternatives', 'https://www.mindbodygreen.com/articles/eco-5-isnt-composter-its-food-waste-recycler', 'https://shop.mindbodygreen.com/products/liver-detox?itc_content=66221', 'https://www.mindbodygreen.com/articles/inflammation-immunity-and-vagus-nerve-exper'

In [35]:
def get_html_content(links, filename="data/mbg_pages.csv"):
    content = []
    failed_links = []

    for link in links:
        time.sleep(random.uniform(2,4))
        response = fetch_verify_url(link)

        if response:
            content.append({'url': link, 'html': response.text})
        else:
            failed_links.append(link)

    print(f"Fetched pages: {len(content)}")
    print(f"Failed URLs: {len(failed_links)}")

    if content:
        output_dir = os.path.dirname(filename)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)

        df = pandas.DataFrame(content)
        # quoting=1 (QUOTE_ALL) est crucial pour le HTML car il contient des virgules et guillemets
        df.to_csv(filename, index=False, encoding='utf-8', quoting=1)
        print(f"File saved : {filename}")
    else:
        print("No content to save.")

    return content, failed_links

content, failed_links = get_html_content(list(ALL_MBG_LINKS[:1000]))

Fetched pages: 25
Failed URLs: 0
File saved : data/mbg_pages.csv
